In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# We assume you uploaded the exercise folder in root Google Drive folder
!cp -r /content/drive/MyDrive/3d-lmnet-pytorch 3d-lmnet-pytorch
os.chdir('/content/drive/MyDrive/3d-lmnet-pytorch')
print('Installing requirements')
!pip install -r requirements.txt

# Make sure you restart runtime when directed by Colab

Mounted at /content/drive
Installing requirements
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.9/670.9 KB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 KB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 73.8 MB/s eta 0:00:00
     

In [1]:
import os
import sys
import torch
os.chdir('/content/3d-lmnet-pytorch/3d-lmnet-pytorch')
sys.path.insert(1, "/content/3d-lmnet-pytorch/3d-lmnet-pytorch")
print('CUDA availability:', torch.cuda.is_available())

CUDA availability: True


In [2]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch
import skimage

In [3]:
torch.cuda.is_available()
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

## ShapeNet Terms and Conditions

In order to be able to use the data, we agree the below terms and conditions:

1. Researcher shall use the Database only for non-commercial research and educational purposes.
2. Princeton University and Stanford University make no representations or warranties regarding the Database, including but not limited to warranties of non-infringement or fitness for a particular purpose.
3. Researcher accepts full responsibility for his or her use of the Database and shall defend and indemnify Princeton University and Stanford University, including their employees, Trustees, officers and agents, against any and all claims arising from Researcher's use of the Database, including but not limited to Researcher's use of any copies of copyrighted 3D models that he or she may create from the Database.
4. Researcher may provide research associates and colleagues with access to the Database provided that they first agree to be bound by these terms and conditions.
5. Princeton University and Stanford University reserve the right to terminate Researcher's access to the Database at any time.
6. If Researcher is employed by a for-profit, commercial entity, Researcher's employer shall also be bound by these terms and conditions, and Researcher hereby represents that he or she is fully authorized to enter into this agreement on behalf of such employer.
7. The law of the State of New Jersey shall apply to all disputes under this agreement.

### Unzip ShapeNet pointcloud zip

In [4]:
!unzip -q ./data/ShapeNet_pointclouds.zip -d ./data

### Download 2D images

In [5]:
!wget http://cvgl.stanford.edu/data2/ShapeNetRendering.tgz -P ./data

--2023-01-19 09:45:37--  http://cvgl.stanford.edu/data2/ShapeNetRendering.tgz
Resolving cvgl.stanford.edu (cvgl.stanford.edu)... 171.64.64.64
Connecting to cvgl.stanford.edu (cvgl.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cvgl.stanford.edu/data2/ShapeNetRendering.tgz [following]
--2023-01-19 09:45:37--  https://cvgl.stanford.edu/data2/ShapeNetRendering.tgz
Connecting to cvgl.stanford.edu (cvgl.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12318245442 (11G) [application/x-gzip]
Saving to: ‘./data/ShapeNetRendering.tgz’

ShapeNetRendering.t 100%[===================>]  11.47G  12.4MB/s    in 17m 19s 

2023-01-19 10:02:56 (11.3 MB/s) - ‘./data/ShapeNetRendering.tgz’ saved [12318245442/12318245442]



In [6]:
!tar -xf ./data/ShapeNetRendering.tgz -C ./data
#!rm /content/term-project/data/ShapeNetRendering.tgz

### Construct ShapeNet dataset

In [4]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train')
val_dataset = ShapeNet('valid')
#overfit_dataset = ShapeNet('overfit')
test_dataset = ShapeNet('test')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}') 
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(val_dataset)}') 
# Get length, which is a call to __len__ function
print(f'Length of test set: {len(test_dataset)}')  

Length of train set: 26271
Length of val set: 8758
Length of test set: 8755


In [5]:

from skimage.measure import marching_cubes

train_sample = train_dataset[1]
print(f'Input images: {train_sample["img"].shape}')  
print(f'Input point cloud: {train_sample["point"].shape}')  

Input images: (3, 128, 128)
Input point cloud: (2048, 3)


### Print output shape of the 2D Encoder model (both variational and normal versions)

In [6]:
from model.model_2d import ImageEncoder
from torchsummary import summary

model2d_variational=ImageEncoder("variational",128)
model2d_variational.cuda()
input_tensor = torch.randn(1,3,128,128)
input_tensor=input_tensor.cuda()
print("input size:",input_tensor.size())


mu,std = model2d_variational(input_tensor)
print("Mu size:",mu.size(),"Std size:",std.size())
summary(model2d_variational,(3,128,128))
model2d_normal=ImageEncoder("normal",128)
model2d_normal.cuda()
latent=model2d_normal(input_tensor)
print("Latent shape:",latent.size())

input size: torch.Size([1, 3, 128, 128])
Mu size: torch.Size([1, 128]) Std size: torch.Size([1, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
              ReLU-2         [-1, 32, 128, 128]               0
            Conv2d-3         [-1, 32, 128, 128]           9,248
              ReLU-4         [-1, 32, 128, 128]               0
            Conv2d-5           [-1, 64, 63, 63]          18,496
              ReLU-6           [-1, 64, 63, 63]               0
            Conv2d-7           [-1, 64, 63, 63]          36,928
              ReLU-8           [-1, 64, 63, 63]               0
            Conv2d-9           [-1, 64, 63, 63]          36,928
             ReLU-10           [-1, 64, 63, 63]               0
           Conv2d-11          [-1, 128, 31, 31]          73,856
             ReLU-12          [-1, 128, 31, 31]               0
 

### Train 2D Encoder model to match the predicted latent space to the output of 3D Encoder of pointclouds

Normal, L1 loss

In [ ]:
from training import train_2d_to_3d

generalization_config = {
    'experiment_name': '/content/drive/MyDrive/3d-lmnet-pytorch/3d-lmnet-pytorch/2d_logs/normal/L1',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'bottleneck': 512,
    "cat":13,
    'batch_size': 32,
    "loss_criterion":"L1",
    "final_layer":"normal",
    "3d_encoder_path":"/content/3d-lmnet-pytorch/3d-lmnet-pytorch/additional_epoches/model_epoch_500.pth",
    'resume_ckpt': None,
    'learning_rate_model':  0.00005,
    'max_epochs': 30,  
    'save_every_n': 5,
    'validate_every_n': 3,
    "autoencoder_bottleneck":512,
    "autoencoder_hidden_size":256,
    "autoencoder_output_size":2048*3,
    "alpha":None,
    "penalty_angle":None,
    "lambda":None


}

train_2d_to_3d.main(generalization_config)

Normal, L2 loss

In [ ]:
from training import train_2d_to_3d

generalization_config = {
    'experiment_name': '/content/drive/MyDrive/3d-lmnet-pytorch/3d-lmnet-pytorch/2d_logs/normal/L2',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'bottleneck': 512,
    "cat":13,
    'batch_size': 32,
    "loss_criterion":"L2",
    "final_layer":"normal",
    "3d_encoder_path":"/content/3d-lmnet-pytorch/3d-lmnet-pytorch/logs/model_epoch_500.pth",
    'resume_ckpt': None,
    'learning_rate_model':  0.00005,
    'max_epochs': 30,  
    'save_every_n': 1,
    'validate_every_n': 5,
    "autoencoder_bottleneck":512,
    "autoencoder_hidden_size":256,
    "autoencoder_output_size":2048*3,
    "alpha":None,
    "penalty_angle":None,
    "lambda":None


}

train_2d_to_3d.main(generalization_config)

Using device: cuda:0
 initial input 512 initial hidden 256 initial output 6144
Epoch 1/30 with iteration 100/19703: CD loss is 0.0033234222792088985.
Epoch 1/30 with iteration 200/19703: CD loss is 0.002913238015025854.
Epoch 1/30 with iteration 300/19703: CD loss is 0.00305278692394495.
Epoch 1/30 with iteration 400/19703: CD loss is 0.0031200689263641834.
Epoch 1/30 with iteration 500/19703: CD loss is 0.0028260741382837296.
Epoch 1/30 with iteration 600/19703: CD loss is 0.002543890615925193.
Epoch 1/30 with iteration 700/19703: CD loss is 0.002885582856833935.
Epoch 1/30 with iteration 800/19703: CD loss is 0.0023992990609258413.
Epoch 1/30 with iteration 900/19703: CD loss is 0.0026208781637251377.
Epoch 1/30 with iteration 1000/19703: CD loss is 0.0023291660472750664.
Epoch 1/30 with iteration 1100/19703: CD loss is 0.00226319907233119.
Epoch 1/30 with iteration 1200/19703: CD loss is 0.002591541036963463.
Epoch 1/30 with iteration 1300/19703: CD loss is 0.0022245284635573626.
Ep

Variational

In [ ]:
from training import train_2d_to_3d

generalization_config = {
    'experiment_name': '/content/drive/MyDrive/3d-lmnet-pytorch/3d-lmnet-pytorch/2d_logs/normal',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'bottleneck': 512,
    "cat":13,
    'batch_size': 32,
    "loss_criterion":"variational",
    "final_layer":"variational",
    "3d_encoder_path":"/content/3d-lmnet-pytorch/3d-lmnet-pytorch/additional_epoches/model_epoch_500.pth",
    'resume_ckpt': None,
    'learning_rate_model':  0.00005,
    'max_epochs': 30,  
    'save_every_n': 5,
    'validate_every_n': 3,
    "autoencoder_bottleneck":512,
    "autoencoder_hidden_size":256,
    "autoencoder_output_size":2048*3,
    "alpha":0.2,
    "penalty_angle":20,
    "lambda":5.5


}

train_2d_to_3d.main(generalization_config)

### Infer pointclouds using trained 2D Encoder and 3D Decoder models

Variational inferences:

In [ ]:
from inference import inference_2d_to_3d 

generalization_config = {
    'experiment_name': '/content/drive/MyDrive/3d-lmnet-pytorch/3d-lmnet-pytorch/2d_logs/normal',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'bottleneck': 512,
    "cat":13,
    'batch_size': 32,
    "loss_criterion":"normal",
    "final_layer":"normal",
    "3d_encoder_path":"/content/3d-lmnet-pytorch/3d-lmnet-pytorch/additional_epoches/model_epoch_500.pth",
    'resume_ckpt': None,
    'learning_rate_model':  0.00005,
    'max_epochs': 30,  
    'print_every_n': 5,
    'visualize_every_n': 5,
    "autoencoder_bottleneck":512,
    "autoencoder_hidden_size":256,
    "autoencoder_output_size":2048*3

}

Normal inferences:

Using L1 in the training

In [ ]:
from inference.infer_2d_to_3d import Inference2DToPointCloudNormal

id=torch.randint(0,len(test_dataset))
val_config={"final_layer" : "normal",
        "bottleneck" : 512,
        "input_size" : None,
        "hidden_size" : None,
        "output_size" : None,
        "bnorm" : True,
        "bnorm_final" : False,
        "regularizer" : None,
        "weight_decay" : 0.001,
        "dropout_prob" : None}
Inference2DToPointCloudNormal(test_dataset[id],"content/3d-lmnet-pytorch/runs/2d_to_3d_normal", "content/3d-lmnet-pytorch/runs/3d_pointcloud_decoder",val_config,device)

Using L2 in the training

In [ ]:
from inference.infer_2d_to_3d import Inference2DToPointCloudNormal

id=torch.randint(0,len(test_dataset))
val_config={"final_layer" : "normal",
        "bottleneck" : 512,
        "input_size" : None,
        "hidden_size" : None,
        "output_size" : None,
        "bnorm" : True,
        "bnorm_final" : False,
        "regularizer" : None,
        "weight_decay" : 0.001,
        "dropout_prob" : None}
Inference2DToPointCloudNormal(test_dataset[id],"content/3d-lmnet-pytorch/runs/2d_to_3d_normal", "content/3d-lmnet-pytorch/runs/3d_pointcloud_decoder",val_config,device)

## 3D Autoencoder Training

In [ ]:
from training import train_ae


    # parser.add_argument("--root", type=str, default="./data")
    # parser.add_argument("--npoints", type=int, default=2048)
    # parser.add_argument("--mpoints", type=int, default=2025)
    # parser.add_argument("--batch_size", type=int, default=16)
    # parser.add_argument("--lr", type=float, default=1e-4)
    # parser.add_argument("--weight_decay", type=float, default=1e-6)
    # parser.add_argument("--epochs", type=int, default=400)
    # parser.add_argument("--num_workers", type=int, default=4)
    # parser.add_argument("--log_dir", type=str, default="./log")

generalization_config = {
    'root': './3d-lmnet-pytorch/',
    'experiment_name': '3d_autoencoder',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'npoints': 2048,
    'mpoints': 2025,
    'lr': 1e-4,
    "autoencoder":"/content/3d-lmnet-pytorch/3d-lmnet-pytorch/additional_epoches/model_epoch_300.pth",
    'weight_decay': 1e-6,
    'bottleneck': 512,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate_model':  0.00005,
    'max_epochs': 200,  
    'num_workers': 4,
    "input_size" : 256,
    "hidden_size" : 256,
    "output_size" : 2048*3,
    'log_dir': '/content/drive/MyDrive/3d-lmnet-pytorch/3d-lmnet-pytorch/additional_epoches',
    'print_every_n': 5,
    'visualize_every_n': 5,
}

train_ae.main(generalization_config)

 initial input 512 initial hidden 16 initial output 3
Begin Training...
initial point cloud size torch.Size([16, 3, 2048])
tensor size after encoder torch.Size([16, 512])
tensor size after decoder torch.Size([16, 3])
recons size torch.Size([16, 3, 1])
point cloud size before loss function torch.Size([16, 2048, 3])
recons size before loss function torch.Size([16, 1, 3])


ValueError: ignored

### 3D Inferences

In [ ]:
from inference import infer_3d

generalization_config = {
    "autoencoder":"/content/drive/MyDrive/3d-lmnet-pytorch/3d-lmnet-pytorch/logs/model_epoch_500.pth",
    'bottleneck': 512,
    'batch_size': 1,
    'num_workers': 4,
    "input_size" : 256,
    "hidden_size" : 256,
    "output_size" : 2048*3,
}

infer_3d.main(generalization_config)